In [ ]:
!pip install transformers[torch]
!pip install  sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 30.3 MB/s 
     |████████████████████████████████| 182 kB 64.8 MB/s 
     |████████████████████████████████| 7.6 MB 47.4 MB/s 


In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel, AutoConfig
import numpy as np
import torch
from torch.nn.functional import normalize
import requests

import torch.nn.functional as F
import torch.nn as nn
import torch

In [ ]:
def get_raw_data():
    train_data_url = "https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/train.json"
    test_data_url = "https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/dev.json"
    train_data_json = requests.get(train_data_url).json()
    test_data_json = requests.get(test_data_url).json()
    return train_data_json, test_data_json

CLASSES = ['PREAMBLE', 'NONE', 'FAC', 'ARG_RESPONDENT', 'RLC', 'ARG_PETITIONER', 'ANALYSIS', 'PRE_RELIED', 'RATIO', 'RPC', 'ISSUE', 'STA', 'PRE_NOT_RELIED']
train_data_json, test_data_json = get_raw_data()

In [ ]:
labels_train=[]
data=[]
labels_test=[]
data_test=[]
for i in train:
  for annotations in i['annotations']:
    for results in annotations['result']:
      data.append(results['value']['text'].strip())
      labels_train.append(results['value']['labels'][0])
for i in dev:
  for annotations in i['annotations']:
    for results in annotations['result']:
      data_test.append(results['value']['text'].strip())
      labels_test.append(results['value']['labels'][0])

In [ ]:
print(len(labels_train))
print(len(labels_test))

28986
2890


In [ ]:
model = SentenceTransformer('sentence-transformers/sentence-t5-large')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/461 [00:00<?, ?B/s]

In [ ]:
def get_embeddings(sentence):
  return model.encode(sentence)

In [ ]:
TRAIN_SIZE = len(labels_train)
print(TRAIN_SIZE)

28986


In [ ]:
X = []
labels = list(set(labels_train))
y = []
progress = 0
for sentence, label in zip(data[:TRAIN_SIZE],
                        labels_train[:TRAIN_SIZE]):
    X.append(get_embeddings(sentence))
    y.append(labels.index(label))
    progress += 1
    if progress % 500 == 0:
        print('Progress Percent = {}%'.format(100 * progress / TRAIN_SIZE))

X_test = []
labels = list(set(labels_test))
y_test = []
progress = 0
for sentence, label in zip(data[:TEST_SIZE],
                        labels_train[:TEST_SIZE]):
    X_test.append(get_embeddings(sentence))
    y_test.append(labels.index(label))
    progress += 1
    if progress % 500 == 0:
        print('Progress Percent = {}%'.format(100 * progress / TEST_SIZE))


In [ ]:
X_test = np.array(X)
y_test = np.array(y)
print(X_test.shape, y_test.shape)

(28986, 768) (28986,)


In [ ]:

class LSTM_Model(nn.Module):

    def __init__(self):
        super(LSTM_Model, self).__init__()

        self.lstm = torch.nn.LSTM(input_size=768, hidden_size=64,
                                                             num_layers=3, batch_first=True, bidirectional=True)
        self.output_layer = nn.Linear(128,13)

    def forward(self, batch):
        lstm_out, _ = self.lstm(batch)
        # lstm_out, _ = self.lstm(embeds)
        # print(lstm_out.shape)
        tag_space = self.output_layer(lstm_out)
        tag_scores = F.softmax(tag_space, dim=1)
        return tag_scores


In [ ]:
model = LSTM_Model()

In [ ]:
print(model)

LSTM_Model(
  (lstm): LSTM(768, 64, num_layers=3, batch_first=True, bidirectional=True)
  (output_layer): Linear(in_features=128, out_features=13, bias=True)
)


In [ ]:
import torch
train_data = []
test_data = []
import random
for i in range(len(X)):
  train_data.append([X[i], y[i]])
for i in range(len(X_test)):
  test_data.append([X_test[i], y_test[i]])

trainloader = torch.utils.data.DataLoader(train_data, shuffle=False, batch_size=64)
testloader = torch.utils.data.DataLoader(test_data, shuffle=False, batch_size=64)

In [ ]:
total=0
correct =0
for i, (input, labels) in enumerate(testloader):
  output = model(input)
  _, predicted = torch.max(output.data, 1)

  total += labels.size(0)
  correct += (predicted == labels).sum()

print(correct/total)

tensor(0.0381)


In [ ]:
num_epochs = 10
# iter = 0
import torch
learning_rate = 0.1
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    for i, (input, labels) in enumerate(trainloader):
        optimizer.zero_grad()
        outputs = model(input)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    correct = 0
    total = 0
    for input, labels in testloader:
      outputs = model(input)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum()

    accuracy = 100 * correct / total

    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))


Epoch: 0. Loss: 2.206331729888916. Accuracy: 39.23875427246094
Epoch: 1. Loss: 2.206331729888916. Accuracy: 39.23875427246094
Epoch: 2. Loss: 2.206331729888916. Accuracy: 39.23875427246094
Epoch: 3. Loss: 2.206331729888916. Accuracy: 39.23875427246094
Epoch: 4. Loss: 2.206331729888916. Accuracy: 39.23875427246094
Epoch: 5. Loss: 2.206331729888916. Accuracy: 39.23875427246094
Epoch: 6. Loss: 2.206331729888916. Accuracy: 39.23875427246094
Epoch: 7. Loss: 2.206331729888916. Accuracy: 39.23875427246094
Epoch: 8. Loss: 2.206331729888916. Accuracy: 39.23875427246094
Epoch: 9. Loss: 2.206331729888916. Accuracy: 39.23875427246094


In [ ]:
for input, labels in testloader:
      outputs = model(input)
      _, predicted = torch.max(outputs.data, 1)
      print(predicted)

In [ ]:
print(labels)

tensor([ 8,  4, 11,  6,  6,  6,  6, 12,  6,  6,  8,  0,  4, 12, 11,  4,  4,  2,
         6, 12,  6,  9, 11,  0,  9,  4,  6,  7, 11,  6,  7,  1, 11,  6,  8,  6,
         6,  9,  6,  1,  4,  3,  6,  6,  9,  6,  4, 12,  6,  4, 12,  4, 12,  4,
         4,  0,  6,  4])


In [ ]:
import plda

In [ ]:
PLDA_classifier = plda.Classifier()
PLDA_classifier.fit_model(np.array(X),
                          np.array(y))

In [ ]:
from keras.layers import Dense, Input, LSTM, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.layers.embeddings import Embedding
from keras.layers.merge import concatenate
from keras.models import load_model
from keras.models import Model

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(13, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


In [ ]:
def predict_doc_at(query):
    query_embedding = get_embeddings(query)
    predictions, log_p_predictions = PLDA_classifier.predict(query_embedding)

    predictions = train_labels_unique[predictions]
    return predictions

In [ ]:
preds_dev=[]
for query in test_data:
    preds = predict_doc_at(query)
    preds_dev.append(preds)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1score=f1_score(labels_all_dev, preds_dev, average="macro")

In [ ]:
f1score

0.4166959405717767

In [ ]:
matches=0
all=0
for i,j in zip(labels_test, preds_dev):
  if i==j:
    matches+=1
  else:
    print("i: "+i)
    print("j: "+ j)
    print("--------------")
  all+=1

i: PREAMBLE
j: NONE
--------------
i: PREAMBLE
j: FAC
--------------
i: PREAMBLE
j: FAC
--------------
i: PREAMBLE
j: PRE_RELIED
--------------
i: PREAMBLE
j: STA
--------------
i: PREAMBLE
j: FAC
--------------
i: PREAMBLE
j: PRE_RELIED
--------------
i: PREAMBLE
j: PRE_RELIED
--------------
i: ANALYSIS
j: NONE
--------------
i: FAC
j: ANALYSIS
--------------
i: FAC
j: RLC
--------------
i: FAC
j: RATIO
--------------
i: FAC
j: RLC
--------------
i: FAC
j: RLC
--------------
i: FAC
j: PRE_RELIED
--------------
i: FAC
j: PRE_RELIED
--------------
i: FAC
j: PRE_RELIED
--------------
i: FAC
j: STA
--------------
i: FAC
j: ARG_PETITIONER
--------------
i: FAC
j: RPC
--------------
i: FAC
j: RPC
--------------
i: FAC
j: ANALYSIS
--------------
i: FAC
j: RLC
--------------
i: RLC
j: ISSUE
--------------
i: ARG_PETITIONER
j: RPC
--------------
i: ANALYSIS
j: ISSUE
--------------
i: ANALYSIS
j: PRE_NOT_RELIED
--------------
i: ANALYSIS
j: PRE_NOT_RELIED
--------------
i: ANALYSIS
j: RATIO
---

In [ ]:
prec=matches/all

In [ ]:
prec

0.5

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from numpy.core.fromnumeric import argmax

In [ ]:
# !conda install -c conda-forge faiss-gpu cudatoolkit=10.1 -y

In [ ]:
import faiss

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
index = faiss.IndexIVFFlat(faiss.IndexFlatL2(X.shape[1])  , X.shape[1], len(train_unique_labels), faiss.METRIC_L2)

In [ ]:
index.train(X)
index.add(X)

False
0
True
28986


In [ ]:
embs_dev=[]

for i in data_dev:
  embs_dev.append(get_embeddings(i))

embs_dev=np.array(embs_dev)
embs_dev.shape

In [ ]:
distances, indices = index.search(embs_dev, 1)

In [ ]:
preds_cos=[]
for i in indices:
    preds_cos.append(train_labels_unique[y[i[0]]])


In [ ]:
from sklearn.metrics import f1_score
f1score=f1_score(labels_all_dev, preds_cos, average="macro")

In [ ]:
f1score

0.3524619251870554

In [ ]:
matches=0
all=0
for i,j in zip(labels_all_dev, preds_cos):
  if i==j:
    matches+=1
  all+=1

In [ ]:
prec=matches/all

In [ ]:
prec

0.4806228373702422